In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

In [ ]:
df = pd.read_csv("../Output/topic_keyword_freq_NDC_sentences_CN.csv")
df['num_words'] = df['text_content'].apply(lambda x: len(x.split()))
df

In [ ]:
grouped = df.groupby(["iso","country", "ndc", "date"]).sum()
grouped

In [ ]:
keywords_df = pd.read_csv("../Data/keywords_for_marine_policy.csv")
keywords = list(set(keywords_df['Keyword']))
topics = list(set(keywords_df['Topic']))

In [ ]:
for topic in topics:
    grouped[topic + "_freq_per_10000"] = (grouped[topic] / grouped["num_words"]) * 10000
for keyword in keywords:
    grouped[keyword + "_freq_per_10000"] = (grouped[keyword] / grouped["num_words"]) * 10000
grouped['keyword_freq_per_10000'] = (grouped[keywords].sum(axis=1) / grouped['num_words']) * 10000
grouped 

In [ ]:
overall_drop = list(set(keywords + topics))
grouped = grouped.drop(columns=overall_drop)
grouped = grouped.drop(columns=['fisheries.1'])
grouped

In [ ]:
grouped = grouped.reset_index()
grouped

In [ ]:
grouped[grouped['iso'] == 'SSD']

In [ ]:
country_to_code = pd.read_csv("https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv")
country_to_code

In [ ]:
country_to_code.astype({"Numeric code": object})
def three_dig_num(num):
    num = str(num)
    if len(num) == 1:
        return "00" + num 
    elif len(num) == 2:
        return "0" + num
    else:
        return num 
country_to_code["Numeric code"] = country_to_code["Numeric code"].apply(three_dig_num)
country_to_code


In [ ]:
cool_dict = {country_to_code.loc[i,"Alpha-3 code"]: country_to_code.loc[i, "Numeric code"] for i in country_to_code.index}
cool_dict["SSD"] = 728
cool_dict

In [ ]:
grouped['numeric_code'] = grouped['iso'].apply(lambda x: cool_dict[x])
grouped

In [ ]:
grouped['date_ndc'] = pd.to_datetime(grouped['date'])
most_recent_ndcs = grouped.loc[grouped.groupby('country')['date_ndc'].idxmax()]
most_recent_ndcs = most_recent_ndcs.drop(columns=["date_ndc"])
most_recent_ndcs

In [ ]:
def in_EU(string):
    return string in ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]

In [ ]:
iso_to_country = {
    "BEL": "Belgium",
    "BGR": "Bulgaria",
    "HRV": "Croatia",
    "CYP": "Cyprus",
    "CZE": "Czech Republic",
    "DNK": "Denmark",
    "EST": "Estonia", 
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "HUN": "Hungary",
    "IRL": "Ireland",
    "ITA": "Italy",
    "LVA": "Latvia", 
    "LTU": "Lithuania", 
    "LUX": "Luxembourg", 
    "MLT": "Malta", 
    "NLD": "Netherlands", 
    "POL": "Poland",
    "PRT": "Portugal",
    "ROU": "Romania", 
    "SVK": "Slovakia",
    "SVN": "Slovenia",
    "ESP": "Spain",
    "SWE": "Sweden"
}

In [ ]:
most_recent_ndcs = most_recent_ndcs.reset_index(drop=True)
most_recent_ndcs

In [ ]:
most_recent_ndcs[most_recent_ndcs['iso'].apply(in_EU)]

In [ ]:
for iso_code in iso_to_country:
    country_name = iso_to_country[iso_code]
    old_row = most_recent_ndcs.loc[51]
    old_row.loc['iso'] = iso_code 
    old_row.loc['country'] = country_name 
    old_row.loc['numeric_code'] = cool_dict[iso_code]
    most_recent_ndcs.loc[len(most_recent_ndcs)] = old_row
most_recent_ndcs

In [ ]:
most_recent_ndcs = most_recent_ndcs.sort_values(by="country").reset_index(drop=True)
most_recent_ndcs

In [ ]:
aut_index = most_recent_ndcs[most_recent_ndcs['iso'] == "AUT"].index[0]
most_recent_ndcs.loc[aut_index, 'country'] = "Austria"
most_recent_ndcs

In [ ]:
most_recent_ndcs[most_recent_ndcs["country"] == "Austria"]

In [ ]:
most_recent_ndcs.to_csv("../Output/topic_keyword_freq_by_countryndc_CN.csv", index=False)